<a href="https://colab.research.google.com/github/thaisja/projeto-aplicado-netflix/blob/main/01_NETFLIX_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# %%
!pip -q install pyarrow fastparquet tqdm

import os, gc, math, random, sys, textwrap, numpy as np, pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

pd.set_option("display.max_columns", 50)
pd.set_option("display.float_format", lambda x: f"{x:,.4f}")
sns.set()
SEED = 42
random.seed(SEED); np.random.seed(SEED)



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 13.9 MB/s eta 0:00:00


In [4]:
from google.colab import drive
drive.mount('/content/drive')

# caminho da pasta no Drive
DATA_DIR = "/content/drive/MyDrive/MACKENZIE/4S Ciência de Dados"

# lista dos arquivos do Netflix Prize que você colocou lá
FILES = [
    f"{DATA_DIR}/combined_data_1.txt",
    f"{DATA_DIR}/combined_data_2.txt",
    f"{DATA_DIR}/combined_data_3.txt",
    f"{DATA_DIR}/combined_data_4.txt"
]

# só para verificar se o Colab encontrou os arquivos
import os
for f in FILES:
    print(f, "->", os.path.exists(f))



Mounted at /content/drive
/content/drive/MyDrive/MACKENZIE/4S Ciência de Dados/combined_data_1.txt -> True
/content/drive/MyDrive/MACKENZIE/4S Ciência de Dados/combined_data_2.txt -> True
/content/drive/MyDrive/MACKENZIE/4S Ciência de Dados/combined_data_3.txt -> True
/content/drive/MyDrive/MACKENZIE/4S Ciência de Dados/combined_data_4.txt -> True


In [ ]:
!pip -q install pyarrow fastparquet tqdm

import random, pandas as pd, numpy as np
from tqdm import tqdm
import gc

SAMPLE_RATE = 0.002   # 0.2% da base total (ajuste conforme sua RAM/tempo)
MAX_ROWS_PER_FILE = None   # ou um inteiro para limitar por arquivo (ex.: 500_000)
SEED = 42
random.seed(SEED)

CACHE_PARQUET = "/content/netflix_all_sample.parquet"  # cache local do Colab
CACHE_PARQUET_DRIVE = f"{DATA_DIR}/netflix_all_sample.parquet"  # opcional: salvar no Drive tb

def iter_netflix_file(path, sample_rate=1.0, max_rows=None, seed=SEED):
    rng = random.Random(seed)
    movie_id = None
    out = 0
    with open(path, "r") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            if line.endswith(":"):
                movie_id = int(line[:-1])
                continue
            # linha do tipo: user_id,rating,date
            try:
                user_id, rating, dt = line.split(",")
                if max_rows is not None and out >= max_rows:
                    break
                if sample_rate < 1.0 and rng.random() > sample_rate:
                    continue
                yield (int(user_id), movie_id, float(rating), dt)
                out += 1
            except:
                continue

def load_all(files, sample_rate=SAMPLE_RATE, max_rows_per_file=MAX_ROWS_PER_FILE):
    rows = []
    for p in files:
        print("Lendo:", p)
        for tup in tqdm(iter_netflix_file(p, sample_rate=sample_rate, max_rows=max_rows_per_file)):
            rows.append(tup)
    df = pd.DataFrame(rows, columns=["user_id","movie_id","rating","date"])
    df["user_id"]  = df["user_id"].astype("int32")
    df["movie_id"] = df["movie_id"].astype("int32")
    df["rating"]   = df["rating"].astype("float32")
    df["date"]     = pd.to_datetime(df["date"], format="%Y-%m-%d", errors="coerce")
    return df

# Cache: tenta carregar, senão cria
try:
    import os
    if os.path.exists(CACHE_PARQUET):
        df = pd.read_parquet(CACHE_PARQUET)
        print("Carregado do cache local:", CACHE_PARQUET, df.shape)
    elif os.path.exists(CACHE_PARQUET_DRIVE):
        df = pd.read_parquet(CACHE_PARQUET_DRIVE)
        print("Carregado do cache no Drive:", CACHE_PARQUET_DRIVE, df.shape)
    else:
        df = load_all(FILES, sample_rate=SAMPLE_RATE, max_rows_per_file=MAX_ROWS_PER_FILE)
        df.to_parquet(CACHE_PARQUET, index=False)
        # opcional: persistir no Drive para reuso entre sessões
        df.to_parquet(CACHE_PARQUET_DRIVE, index=False)
        print("Amostra combinada salva em:", CACHE_PARQUET, "e", CACHE_PARQUET_DRIVE, df.shape)
except Exception as e:
    print("Erro ao carregar/criar cache:", e)

df.head()


Lendo: /content/drive/MyDrive/MACKENZIE/4S Ciência de Dados/combined_data_1.txt


47911it [00:19, 2508.95it/s]


Lendo: /content/drive/MyDrive/MACKENZIE/4S Ciência de Dados/combined_data_2.txt


37911it [00:19, 2514.23it/s]